In [ ]:
import streamlit as st
import pandas as pd
import folium
from folium.plugins import MarkerCluster, HeatMap
from streamlit_folium import st_folium

In [ ]:



# Page title
st.set_page_config(page_title='Permit Data Exploration', page_icon='📊')
st.title('📊 Permit Data Exploration')

with st.expander('About this app'):
    st.markdown('**What can this app do?**')
    st.info('Interact with the permit dataset/Parquet files and explore each column.')
    st.markdown('**How to use the app?**')
    st.warning('This is a prototype, functionality will be added as we develop further.')

st.subheader('Explore Permit Data by Construction Type and State')

In [ ]:

# File upload
uploaded_file = st.file_uploader("Upload your Parquet file", type=['parquet'])

if uploaded_file is not None:
    df = pd.read_parquet(uploaded_file)
else:
    df = pd.read_parquet('data/csv_reveal-gc-2020-50.parquet')

In [ ]:

# Input widgets
## Construction Type selection
const_type_list = df.CONST_TYPE.unique()
const_type_selection = st.multiselect('Select Construction Types', const_type_list, const_type_list[:3])


In [ ]:

## State selection
state_list = df.SITE_STATE.unique()
state_selection = st.multiselect('Select States', state_list, state_list[:3])

jurisdiction_list = df['SITE_JURIS'].unique().tolist()
jurisdiction_selection = st.multiselect('Select Jurisdictions', jurisdiction_list, jurisdiction_list[:3])


In [ ]:

# Filter data based on selections
df_selection = df[
    df['CONST_TYPE'].isin(const_type_selection) &
    df['SITE_STATE'].isin(state_selection) &
    df['SITE_JURIS'].isin(jurisdiction_selection)
]


In [2]:

import streamlit as st
import pandas as pd
import numpy as np
import altair as alt

if "data" not in st.session_state:
    st.session_state.data = pd.DataFrame(
        np.random.randn(20, 3), columns=["a", "b", "c"]
    )
df = st.session_state.data

point_selector = alt.selection_point("point_selection")
interval_selector = alt.selection_interval("interval_selection")
chart = (
    alt.Chart(df)
    .mark_circle()
    .encode(
        x="a",
        y="b",
        size="c",
        color="c",
        tooltip=["a", "b", "c"],
        fillOpacity=alt.condition(point_selector, alt.value(1), alt.value(0.3)),
    )
    .add_params(point_selector, interval_selector)
)

event = st.altair_chart(chart, key="alt_chart", on_select="rerun")

event



2024-06-03 17:13:05.372 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2024-06-03 17:13:05.722 
  command:

    streamlit run /home/vscode/.local/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


{'selection': {'interval_selection': {}, 'point_selection': {}}}

In [ ]:
df_selection.columns

In [ ]:
df_selection[['PMT_DATE','CREATEDATE','PMT_WEEK']]

In [ ]:
df_selection['PMT_DATE']

In [ ]:
df_selection['SITE_GEOCODE_DATE']

In [ ]:

# Display DataFrame
st.dataframe(df_selection)
# Filter data based on selections
df_selection = df[df.CONST_TYPE.isin(const_type_selection) & df.SITE_STATE.isin(state_selection)]

# Display DataFrame
st.dataframe(df_selection)


In [ ]:

# Pivot table to aggregate data
reshaped_df = df_selection.pivot_table(index='SITE_STATE', columns='CONST_TYPE', values='PERMITID', aggfunc='count', fill_value=0)
reshaped_df = reshaped_df.sort_values(by='SITE_STATE', ascending=False)


In [ ]:

# Display reshaped DataFrame
st.subheader('Aggregated Data by State and Construction Type')
st.dataframe(reshaped_df)


In [ ]:

# Prepare data for chart
df_chart = reshaped_df.reset_index().melt(id_vars='SITE_STATE', var_name='CONST_TYPE', value_name='COUNT')


In [ ]:
st.subheader('Monthly Lag Analysis')


# Subtract 7 days for the grace period
df_selection['CREATEDATE']  =pd.to_datetime(df_selection['CREATEDATE'],format = "%m/%d/%Y")
df_selection['PMT_DATE']  = pd.to_datetime(df_selection['PMT_DATE'],format = "%m/%d/%Y")
# Calculate month lag

df_selection['MONTH_LAG'] = (df_selection['CREATEDATE']) -(df_selection['PMT_DATE'])